In [ ]:
!pip install --upgrade py2neo
!pip install --upgrade neo4jupyter
!pip install --upgrade neo4j

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

#permite conectarse a la BD
#Fuente: https://py2neo.org/2021.1/
import py2neo
from py2neo import Graph,Node,Relationship

#permite representar gráficamente los grafos obtenidos
#Fuente: https://github.com/merqurio/neo4jupyter
import neo4jupyter
neo4jupyter.init_notebook_mode()


print ('py2neo version', py2neo.__version__)


In [ ]:
# url de la BD
dburl='neo4j+s://aec80664.databases.neo4j.io'

#indica usuario y pass a la BD
user="neo4j"
pasw="GDMOXPUVWbD6QVHRrT_VonosnbfEAXW3AF77QjLqZPw"

graph = Graph(dburl, auth=(user, pasw))

from pandas import read_csv,isnull
from neo4j import GraphDatabase

uri             = dburl # url of your neo4j proyect, for example "bolt://localhost:7687"

userName        = user # username

password        = pasw # user password

# Connect to the neo4j database server

graphDB_Driver  = GraphDatabase.driver(uri, auth=(userName, password))



## Importación de datos


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive') # CHANGE YOUR DATA SOURCE

In [ ]:

file_noc = 'drive/MyDrive/teams.csv'

# LOAD TEAMS + ARENA

with graphDB_Driver.session() as graphDB_Session:
    df = read_csv(file_noc,low_memory=False)
    largo = len(df.index)
    print(largo)
    for idx in tqdm(range(0,largo)):
        # get the information of each row
        wanted_df_slice = df.iloc[[idx]]
        team_id = str(wanted_df_slice.TEAM_ID.values[0])
        name = str(wanted_df_slice.NICKNAME.values[0])
        arena = str(wanted_df_slice.ARENA.values[0])
        # cypher query to create the players nodes and their relationship "PLAYED" that matches each player with the games they played and their stats
        cqlNodeQuery  = 'MERGE (t: Team {id: \'' + team_id +'\', name: \'' + name +'\' })'
        cqlNodeQuery += 'MERGE (a: Arena {id: \''+ arena +'\'})'
        cqlNodeQuery += 'MERGE (t)-[:HAS_ARENA]->(a)'

        # Query the graph
        nodes = graphDB_Session.run(cqlNodeQuery)

In [ ]:

file_noc = 'drive/MyDrive/ranking.csv'

# LOAD CONFERENCES and relationships

with graphDB_Driver.session() as graphDB_Session:
    df = read_csv(file_noc,low_memory=False)
    largo = len(df.index)
    teams=set()
    for idx in range(0,largo):
        # get the information of each row
        wanted_df_slice = df.iloc[[idx]]
        team_id = str(wanted_df_slice.TEAM_ID.values[0])
        conference = str(wanted_df_slice.CONFERENCE.values[0])
        # cypher query to create the players nodes and their relationship "PLAYED" that matches each player with the games they played and their stats
        cqlNodeQuery = 'MATCH (t:Team {id: \''+ team_id +'\'}) '
        cqlNodeQuery += 'MERGE (conf:Conference {id: \''+ conference +'\'})'
        cqlNodeQuery += 'MERGE (t)-[:BELONGS]->(conf)'
        if len(teams) < 30 :
          if team_id not in teams: # do not add team to conference twice
            nodes = graphDB_Session.run(cqlNodeQuery)
            teams.add(team_id)
        else:
          break;


In [ ]:
file_noc = 'games.csv'

with graphDB_Driver.session() as graphDB_Session:
    df = read_csv(file_noc,low_memory=False)
    largo = len(df.index) #
    with graphDB_Session.begin_transaction() as tx:
      for idx in tqdm(range(0,largo)):
          # get the information of each row
          wanted_df_slice = df.iloc[[idx]]

          game_id = str(wanted_df_slice.GAME_ID.values[0])
          home_team_wins = bool(wanted_df_slice.HOME_TEAM_WINS.values[0])
          home_team = str(wanted_df_slice.HOME_TEAM_ID.values[0])
          away_team = str(wanted_df_slice.VISITOR_TEAM_ID.values[0])
          winner_team = home_team if home_team_wins else away_team
          loser_team = away_team if home_team_wins else home_team
          season = str(wanted_df_slice.SEASON.values[0])
          date = str(wanted_df_slice.GAME_DATE_EST.values[0])
          #home team data

          home_pts = "0" if isnull(wanted_df_slice.PTS_home.values[0]) else str(wanted_df_slice.PTS_home.values[0])
          home_fg_pct = "0" if isnull(wanted_df_slice.FG_PCT_home.values[0]) else str(wanted_df_slice.FG_PCT_home.values[0])
          home_ft_pct = "0" if isnull(wanted_df_slice.FT_PCT_home.values[0]) else str(wanted_df_slice.FT_PCT_home.values[0])
          home_fg3m_pct = "0" if isnull(wanted_df_slice.FG3_PCT_home.values[0]) else str(wanted_df_slice.FG3_PCT_home.values[0])
          home_reb = "0" if isnull(wanted_df_slice.REB_home.values[0]) else str(wanted_df_slice.REB_home.values[0])
          home_ast = "0" if isnull(wanted_df_slice.AST_home.values[0]) else str(wanted_df_slice.AST_home.values[0])

          #away team data

          away_pts = "0" if isnull(wanted_df_slice.PTS_away.values[0]) else str(wanted_df_slice.PTS_away.values[0])
          away_fg_pct = "0" if isnull(wanted_df_slice.FG_PCT_away.values[0]) else str(wanted_df_slice.FG_PCT_away.values[0])
          away_ft_pct = "0" if isnull(wanted_df_slice.FT_PCT_away.values[0]) else str(wanted_df_slice.FT_PCT_away.values[0])
          away_fg3m_pct = "0" if isnull(wanted_df_slice.FG3_PCT_away.values[0]) else str(wanted_df_slice.FG3_PCT_away.values[0])
          away_reb = "0" if isnull(wanted_df_slice.REB_away.values[0]) else str(wanted_df_slice.REB_away.values[0])
          away_ast = "0" if isnull(wanted_df_slice.AST_away.values[0]) else str(wanted_df_slice.AST_away.values[0])

        # cypher query to create the relationship "HOME_TEAM" that matches the local team stats of each game
          full_query  = 'MATCH (ht:Team {id: \''+ home_team +'\'}) MATCH (at:Team {id: \''+ away_team+'\'})'
          full_query  += ' MERGE (g: Game {id: \'' + game_id +'\', date: DATE(\'' + date+'\')}) MERGE (tsh: TeamSeason {id:\''+home_team+season+'\'}) MERGE (ht)-[:_'+season+']->(tsh)'
          full_query  += ' MERGE (g)-[r:HOME_TEAM {PTS: '+ home_pts +', FG_PCT: '+ home_fg_pct +', FT_PCT: '+home_ft_pct+', AST: '+home_ast+', REB: '+home_reb+' }]->(tsh)'
          full_query += ' MERGE (s:Season {id: \'' + season +'\'}) MERGE (tsh)-[:SEASON]->(s)'
          # cypher query to create the relationship "AWAY_TEAM" that matches the away team stats of each game
          full_query += ' MERGE (tsa: TeamSeason {id:\''+away_team+season+'\'}) MERGE (at)-[:_'+season+']->(tsa)'
          full_query += ' MERGE (g)-[o:AWAY_TEAM {PTS: '+ away_pts +', FG_PCT: '+ away_fg_pct +', FT_PCT: '+away_ft_pct+', AST: '+away_ast+', REB: '+away_reb+' }]->(tsa)'
          full_query += ' MERGE (tsa)-[:SEASON]->(s)'
          # add winner and loser
          full_query +=' MERGE (g)-[:WINNER]->(tsh) MERGE (g)-[:LOSER]->(tsa)' if home_team_wins else ' MERGE (g)-[:WINNER]->(tsa) MERGE (g)-[:LOSER]->(tsh)'


          # Season query
          tx.run(full_query)
          if idx % 100 == 0:
             tx.commit()
             tx = graphDB_Session.begin_transaction()

      tx.commit()


In [ ]:

file_noc = 'drive/MyDrive/games_details.csv'
time_dict = {}
with graphDB_Driver.session() as graphDB_Session:
    df = read_csv(file_noc,low_memory=False)
    largo = len(df.index)
    print(largo)

    with graphDB_Session.begin_transaction() as tx:
      # 68501
      for idx in tqdm(range(268501,largo)): # Re-run from new range if fails
          # get the information of each row
          wanted_df_slice = df.iloc[[idx]]
          game_id = str(wanted_df_slice.GAME_ID.values[0])
          name = '\"'+str(wanted_df_slice.PLAYER_NAME.values[0])+'\"'
          player_id = str(wanted_df_slice.PLAYER_ID.values[0])
          team_id = str(wanted_df_slice.TEAM_ID.values[0])
          time_played = str(wanted_df_slice.MIN.values[0])
          secs = 0 # defalut value that represents "Do not play"
          # if Min column has some value, we need to calculate the seconds and store it
          if not isnull(wanted_df_slice.MIN.values[0]) :
              # we split in minutes and seconds
              mins_slice = time_played.split(":")
              # if play at least 1 minute, we need to calculate that time in seconds
              if len(mins_slice) > 1 : # there is wrong data of "-5" seconds played by some players
                  secs = int(float(mins_slice[0]))*60+int(float(mins_slice[1]))

          fgm = "0" if isnull(wanted_df_slice.FGM.values[0]) else str(wanted_df_slice.FGM.values[0])
          fga = "0" if isnull(wanted_df_slice.FGA.values[0]) else str(wanted_df_slice.FGA.values[0])
          fg3a = "0" if isnull(wanted_df_slice.FG3A.values[0]) else str(wanted_df_slice.FG3A.values[0])
          fg3m = "0" if isnull(wanted_df_slice.FG3M.values[0]) else str(wanted_df_slice.FG3M.values[0])
          ftm = "0" if isnull(wanted_df_slice.FTM.values[0]) else str(wanted_df_slice.FTM.values[0])
          fta = "0" if isnull(wanted_df_slice.FTA.values[0]) else str(wanted_df_slice.FTA.values[0])
          oreb = "0" if isnull(wanted_df_slice.OREB.values[0]) else str(wanted_df_slice.OREB.values[0])
          dreb = "0" if isnull(wanted_df_slice.DREB.values[0]) else str(wanted_df_slice.DREB.values[0])
          reb = "0" if isnull(wanted_df_slice.REB.values[0]) else str(wanted_df_slice.REB.values[0])
          ast = "0" if isnull(wanted_df_slice.AST.values[0]) else str(wanted_df_slice.AST.values[0])
          stl = "0" if isnull(wanted_df_slice.STL.values[0]) else str(wanted_df_slice.STL.values[0])
          blk = "0" if isnull(wanted_df_slice.BLK.values[0]) else str(wanted_df_slice.BLK.values[0])
          to = "0" if isnull(wanted_df_slice.TO.values[0]) else str(wanted_df_slice.TO.values[0])
          pf = "0" if isnull(wanted_df_slice.PF.values[0]) else str(wanted_df_slice.PF.values[0])
          pts = "0" if isnull(wanted_df_slice.PTS.values[0]) else str(wanted_df_slice.PTS.values[0])
          # cypher query to create the players nodes and their relationship "PLAYED" that matches each player with the games they played and their stats
          cqlNodeQuery  = 'MATCH (g: Game {id: \'' + game_id +'\' })'
          cqlNodeQuery += 'MERGE (p: Player {id: \''+ player_id +'\', name: '+ name +'})'

          cqlNodeQuery += 'MERGE (p)-[r:PLAYED {team: \''+ team_id +'\', time_played:'+ str(secs) +', FGM: '+fgm+', FGA: '+fga+', FG3A: '+fg3a+',FG3M: '+fg3m+',FTM: '+ftm+', FTA: '+fta+', OREB: '+oreb+',DREB: '+dreb+', REB: '+reb+', AST: '+ast+',STL: '+stl+', BLK: '+blk+', FOULS: '+to+', PF: '+pf+', PTS: '+pts+' }]->(g)'
          cqlNodeQuery +=' SET g.time_played = COALESCE(g.time_played, 0) +'+str(secs/10)


          # Query the graph
          tx.run(cqlNodeQuery)
          if idx % 100 == 0:
             tx.commit()
             last_loaded_row = idx
             tx = graphDB_Session.begin_transaction()

      tx.commit()


# Análisis de estadios

In [ ]:
# GAME COUNT Y ESTADÍSTICAS POR ESTADIO LEBRON JAMES, RÉPLICA GRUPO8
old_query="""MATCH (p:Player {name: "LeBron James"})-[pl:PLAYED]->(g:Game)
MATCH (g)-[w:WINNER]->(t:Team)
WHERE t.id = pl.team
RETURN g.arena AS Arena,SUM(pl.PTS) as Total_PTS, COUNT(w) as WINS
ORDER BY Total_PTS DESC
"""
new_query="""  MATCH (p:Player {name: "LeBron James"})-[pl:PLAYED]-(g:Game)
    MATCH (g)-[w:WINNER]-(ts:TeamSeason)
    MATCH (g)-[:HOME_TEAM]-()-[]-(t:Team)-[:HAS_ARENA]-(a:Arena)
    WHERE substring(ts.id, 0, 10) = pl.team
    RETURN a.id AS Arena, sum(pl.PTS) as PTS, sum(pl.AST) as AST, sum(pl.REB) as REB, count(w) as wins
    ORDER BY PTS DESC
"""
#evaluamos las consultas
start = time()
result=graph.run(query).to_data_frame()
print("Elapsed: ",time() - start)
result



In [ ]:
# Ventaja de localidad para cada estadio/equipo.
season_query='2022'

query=f"""
    MATCH (t:Team)-[:_2021]->(ts:TeamSeason)-[:HOME_TEAM]-(g:Game)
        OPTIONAL MATCH (ts)-[w:WINNER]-(g)
        OPTIONAL MATCH (ts)-[l:LOSER]-(g)
        MATCH (t:Team)-[]->(a:Arena)
        WITH t.name as Team,a.id as Arena, count(g) AS gameCount,count(w) as winsLocal, count(l) as winsAway
        ORDER BY gameCount DESC
        RETURN Arena, Team, Game_count,Wins_Local,Wins_Away
      """

start = time()
result=graph.run(query).to_data_frame()
print("Elapsed: ",time() - start)
result




# Análisis de temporadas

---



In [ ]:
# CONSULTA POR TEAMSEASON

nodos="""MATCH (t:Team)-[:_2015]-(ts:TeamSeason)-[w:WINNER]-(:Game)
WITH t, ts, COUNT(w) as Wins
MATCH (ts)-[l:LOSER]-(:Game)
WITH t.name as Team, ts, Wins, COUNT(l) as Loses
RETURN Team, Wins, Loses, (toFloat(Wins)/(Wins + Loses)) as WinRate
ORDER BY WinRate DESC"""

nodos=graph.run(nodos).to_data_frame()

nodos

# Análisis de conferencias




In [ ]:
# CUÁNTOS PARTIDOS SE JUGARON NORMALMENTE Y CUANTOS FUERON A OVERTIME
season='2021'
query=f"""Match (g:Game)-[]-(:TeamSeason)-[]-(s:Season)
         WHERE s.id ='{season}'
        RETURN DISTINCT ROUND(g.time_played) as Time_played,count(g) order by Time_played """

#evaluamos las consultas
nodos=graph.run(query).to_data_frame()

nodos


In [ ]:
# CUÁNTOS PARTIDOS SE JUGARON NORMALMENTE Y CUANTOS FUERON A OVERTIME DADA UNA SEASON SEPARANDO POR CONFERENCIA
season='2021'

query=f""" Match (s:Season)-[]-(ht:TeamSeason)-[:HOME_TEAM]-(g:Game)-[:AWAY_TEAM]-(at:TeamSeason)-[]-(:Team)-[]-(c:Conference) where c.id='East' and s.id='{season}'
MATCH (ht)-[]-(:Team)-[:BELONGS]-(c)
return DISTINCT ROUND(g.time_played) as Time_played,count(g.time_played) as Game_count order by Time_played"""

#evaluamos las consultas
nodos=graph.run(query).to_data_frame()
print('East')
print(nodos)


query=f"""Match (s:Season)-[]-(ht:TeamSeason)-[:HOME_TEAM]-(g:Game)-[:AWAY_TEAM]-(at:TeamSeason)-[]-(:Team)-[]-(c:Conference) where c.id='West' and s.id='{season}'
MATCH (ht)-[]-(:Team)-[:BELONGS]->(c)
return DISTINCT ROUND(g.time_played) as Time_played,count(g.time_played) as Game_count order by Time_played """

#evaluamos las consultas
nodos=graph.run(query).to_data_frame()
print('West')
print(nodos)


In [ ]:
# Conferencia conferencia mejor q otra por wins/loses
season='2021'

nodos=f"""MATCH (s:Season)-[]-(tEast:TeamSeason)-[:HOME_TEAM|AWAY_TEAM]-(g:Game)-[:HOME_TEAM|AWAY_TEAM]-(tWest:TeamSeason)-[]-(:Team)-[]-(cWest:Conference)
 WHERE cWest.id='West' and s.id='{season}'
 MATCH (tEast)-[]-(:Team)-[:BELONGS]-(cEast:Conference) where cEast.id='East'
 OPTIONAL MATCH (g)-[eastWins:WINNER]-(tEast)
 RETURN count(g) as Game_count, count(eastWins) as East_wins, count(g)-count(eastWins) as West_wins"""

#evaluamos las consultas
nodos=graph.run(nodos).to_data_frame()

nodos

# Análisis de Jugadores

In [ ]:
# Racha de jugadores
season='2021'

nodos="""
  MATCH (p:Player)-[r:PLAYED]-(g:Game)-[:WINNER]-(ts:TeamSeason)-[]-(winningTeam:Team)
  WHERE r.time_played>0
  MATCH (ts)-[]-(s:Season) where s.id='2021'
  WITH p, g, r,winningTeam
  ORDER BY g.date ASC
  WITH p, collect(r.team = winningTeam.id) AS hasWon

  WITH p, reduce(
    acc = [0], outcome IN hasWon |
   CASE
    WHEN outcome THEN [acc[0] + 1] + acc[1..size(acc)]
    ELSE [0] + acc
  END
  ) AS streaks
  UNWIND streaks as streak
  RETURN p.name AS playerName, max(streak) AS longestWinningStreak
  ORDER BY longestWinningStreak DESC
  """

#Edmond Sumner
#evaluamos las consultas
nodos=graph.run(nodos).to_data_frame()

nodos

## Extensión de noción de distancia (rival / compañero)




In [ ]:
# Distancia rival
nodos="""
  MATCH path = (start:Player {name: 'Esteban Batista'})-[:PLAYED*2..6]-(end:Player {name: 'LeBron James'})
  WITH start, end, path, nodes(path) AS nodes, relationships(path) AS rels
  WHERE all(i IN range(1, size(nodes)-1) WHERE
    (labels(nodes[i])[0] = 'Player') OR (labels(nodes[i])[0] = 'Game' AND (rels[i-1]).team <> (rels[i]).team)
  )
  RETURN path,length(path)
  ORDER BY length(path) ASC
  LIMIT 1
"""


nodos=graph.run(nodos).to_data_frame()

nodos

In [ ]:
# Distancia aliada
nodos="""
  MATCH path = (start:Player {name: 'Romeo Langford'})-[:PLAYED*2..6]-(end:Player)
  WITH start, end, path, nodes(path) AS nodes, relationships(path) AS rels
  WHERE all(i IN range(1, size(nodes)-1) WHERE
    (labels(nodes[i])[0] = 'Player') OR (labels(nodes[i])[0] = 'Game' AND (rels[i-1]).team = (rels[i]).team)
  )
  RETURN path,length(path)
  ORDER BY length(path) ASC
  LIMIT 1
"""


nodos=graph.run(nodos).to_data_frame()

nodos